# Lab 3

## Conrad Appel & Eric Hawkins

In [ ]:
%matplotlib inline
import numpy as np
from scipy import ndimage
import os 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as p
from skimage.feature import match_template
import _pickle as cPickle

numlabels = 2

###
# Download dataset from http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
# save all files to ./lab3/imgs/
###

In [ ]:
dics = []
for root, directory, files in os.walk('imgs'):
    for f in files:
        if 'data_batch' in f:
            with open(root+'/'+f, 'rb') as fo:
                dics.append(cPickle.load(fo, encoding='latin1'))
total_imgs = []
for dic in dics:
    for i in range(len(dic['data'])):
        curpic = []
        curpic.append(dic['data'][i]) # 1D img (1024 R, 1024 G, 1024 B)
        curpic.append(dic['labels'][i]) # int representing the label
        total_imgs.append(curpic)

total_imgs = np.array(total_imgs)
imgs_df = p.DataFrame(total_imgs)
imgs_df.columns = ['oneDColor', 'label']

In [ ]:
def toGrayscale(img):
    oneDgray = np.zeros(1024)
    for j in range(1024):
        oneDgray[j] = 0.299*img[j] + 0.587*img[j+1024] + 0.114*img[j+2048]
    return oneDgray

imgs_df['oneDGray'] = imgs_df['oneDColor'].apply(toGrayscale)

In [ ]:
f, axes = plt.subplots(1,10)
for label in range(numlabels):
    axes[label].axis('off')
    axes[label].imshow(imgs_df.loc[lambda df: df.label == label]['oneDGray'].sample(n=1).values[0].reshape((32,32)), cmap=plt.cm.gray)

## Linear Dimensionality Reduction via PCA

In [ ]:
# Principal Components Analysis
pcas = []
for label in range(numlabels):
    x = imgs_df.loc[lambda df: df.label == label]
    
    pixels_are_cols = np.zeros(shape=(len(x), 1024))
    for i in range(len(x)):
        cur_pic = x['oneDGray'].values[i]
        for j in range(1024):
            pixels_are_cols[i][j] = cur_pic[j]
    
    x = pixels_are_cols
    y = [True]*len(x)
    n_comps = 175

    pca = PCA(n_components=n_comps)
    X_pca = pca.fit(x)
    pcas.append((x, pca, X_pca))

In [ ]:
f, axes = plt.subplots(1, numlabels)
for label in range(numlabels):
    axis = axes[label] if numlabels > 1 else axes
    eigenpics = pcas[label][1].components_.reshape((n_comps, 32, 32))
    np.random.shuffle(pcas[label][0])
    recd = pca.inverse_transform(pca.transform(pcas[label][0][0].reshape(1, -1)))
    axis.axis('off')
    axis.imshow(recd.reshape((32,32)), cmap=plt.cm.gray)